In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/stroke.csv')
df.head(10)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


### 더미를 만들어보자

In [3]:
# 흡연 여부 더미로 변환
S_s = pd.get_dummies(df['smoking_status'])

In [4]:
# 결혼여부 더미로 변환
EM = pd.get_dummies(df['ever_married'])

In [5]:
df_D = pd.concat([df,S_s,EM], axis = 1)

### 결측치를 찾아보자(BMI)

In [6]:
# 독립변수와 종속변수 데이터 분리
X = df_D.drop(['stroke'], axis =1)
Y = df_D['stroke']

In [7]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
Train_X.shape

(3832, 17)

In [8]:
# 컬럼별 결측치 확인
Train_X.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  155
smoking_status         0
Unknown                0
formerly smoked        0
never smoked           0
smokes                 0
No                     0
Yes                    0
dtype: int64

In [9]:
# BMI 수치는 개인적인 정보로 대표값대체는 힘들 것이라 판단 - > 결측치 제거
Train_X.dropna(inplace = True)
Train_X.head()

<ipython-input-9-4106350fa820>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train_X.dropna(inplace = True)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,Unknown,formerly smoked,never smoked,smokes,No,Yes
1117,9415,Female,69.0,0,0,Yes,Self-employed,Urban,80.85,29.3,formerly smoked,0,1,0,0,0,1
692,12593,Female,18.0,0,0,No,Private,Urban,80.33,19.7,never smoked,0,0,1,0,1,0
488,28265,Female,42.0,0,0,Yes,Self-employed,Rural,79.14,25.0,formerly smoked,0,1,0,0,0,1
1612,38094,Male,15.0,0,0,No,Self-employed,Urban,68.40,23.0,never smoked,0,0,1,0,1,0
3733,44001,Female,39.0,0,0,Yes,Private,Urban,55.28,31.5,Unknown,1,0,0,0,0,1


In [10]:
Test_X.dropna(inplace = True)

<ipython-input-10-83789ee89ba6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_X.dropna(inplace = True)


### 이상치를 탐지하고 삭제해보자(혈당 수치)

In [17]:
# 이상치 탐지 함수 생성
def IQR_V(val_list) :
    Q1 = np.quantile(val_list, 0.25)
    Q3 = np.quantile(val_list, 0.75)
    IQR = Q3 - Q1
    
    not_outlier_condition = (Q3 + 1.5 * IQR > val_list) & (Q1 - 1.5 * IQR < val_list)
    return not_outlier_condition    

In [27]:
# 혈당 수치 이상치 탐지
cond = Train_X[['avg_glucose_level']].apply(IQR_V)
cond

,avg_glucose_level
1117,True
692,True
488,True
1612,True
3733,True
...,...
1668,True
986,True
1883,True
3890,True


In [29]:
# 이상치 여부확인??
cond.sum(axis = 1) == len(Train_X[['avg_glucose_level']].columns)

1117     True
692      True
488      True
1612     True
3733     True
        ...  
1668     True
986      True
1883     True
3890     True
2685    False
Length: 3677, dtype: bool

In [30]:
# 이상치가 포함 된 행 제거??
Train_X = Train_X.loc[cond.sum(axis = 1) == len(Train_X[['avg_glucose_level']].columns)]
Train_X.shape

(3251, 17)